In [20]:
from Cluster import *
from ClusterManager import *
from Sign import *
from SignManager import *
from Camera import *
from decimal import *
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import csv
import argparse
import math
import pandas as pd

"""
This is the test bed that we will use to test different algorithms.
The code is broken down into the following sections:

1. Command line arguements and initialization
    - This code allows this test bed to be run with different inputs and eventually with different algorithms. The idea is to be able to quickly
    swap between different cluster and classifying combinations in order to do quick analysis.
    - This is where all the files are read in and the data can be formatted (in our case, we round all x, y, and z to 3 decimal places. Yes this is slow. Hopefully we can make this round fn faster).

2. Algorithm Implementation - There are three parts to this part
    a) Filtering
    b) Clustering
    c) Classifying

    It is important to understand the purpose of each section AND it's inputs and outputs.

    Filtering
        - There are no inputs and outputs to filtering. We simply filter the dataframe by columns. We can filter by any column value.

    Clustering
        - The algorithms that are implemented in this section should be defined in the ClusterManager class.
        - Each clustering algorithm should take in a pandas dataframe and should return a list of clusters.
        - A cluster manager object will hold the list of clusters returned from the clustering algorithm. Look in the section below for details.

    Classifying
        - This section is to implement algorithms that will determine if a cluster is a sign and the sign's shape/classification.
        - The input for a classfying algorithm is a list of clusters and the output is a list of signs.
        - A sign manager object will hold the list of signs returned from the classifying algorithm. Look in the section below for details.

3. Book-keeping
    - We do other little things here like:
        - pair a picture to which sign we think it is
        - Writing results out to a file.

"""

"\nThis is the test bed that we will use to test different algorithms.\nThe code is broken down into the following sections:\n\n1. Command line arguements and initialization\n    - This code allows this test bed to be run with different inputs and eventually with different algorithms. The idea is to be able to quickly\n    swap between different cluster and classifying combinations in order to do quick analysis.\n    - This is where all the files are read in and the data can be formatted (in our case, we round all x, y, and z to 3 decimal places. Yes this is slow. Hopefully we can make this round fn faster).\n\n2. Algorithm Implementation - There are three parts to this part\n    a) Filtering\n    b) Clustering\n    c) Classifying\n\n    It is important to understand the purpose of each section AND it's inputs and outputs.\n\n    Filtering\n        - There are no inputs and outputs to filtering. We simply filter the dataframe by columns. We can filter by any column value.\n\n    Cluste

In [22]:
# Rounding Function
def round_half_up(n, decimals=3):
    multiplier = 10 ** decimals
    return math.floor(n*multiplier + 0.5) / multiplier

In [23]:
# Reading in data into a dataframe and formatting it.
df = pd.read_csv('/Volumes/T7 Touch/School/VIP/Data/2015/LiDAR/Routes/Route2015_1/RoutePoints.txt', sep = " ")
df = df.astype({'ID': int})
# TODO: Figure out how to read in data with only a certain number of demicals. This takes a lot of time.
df['Easting'] = df['Easting'].apply(round_half_up)
df['Northing'] = df['Northing'].apply(round_half_up)
df['Altitude'] = df['Altitude'].apply(round_half_up)
df['Retro'] = df['Retro'].apply(round_half_up)

In [24]:
print('Start Filtering')
# Filtering by Retro values
df = df.loc[df['Retro'] > 0.7]
df = df.reset_index(drop=True)
print('End Filtering')

Start Filtering
End Filtering


In [25]:
print('Start Clustering')
cluster_manager = ClusterManager()
cluster_manager.cluster_list = cluster_manager.progressive_kdmean(df)
print('End Clustering')

Start Clustering
End Clustering


In [26]:
print('Start Classifying')
sign_manager = SignManager()
sign_manager.sign_list = sign_manager.num_points(cluster_manager.cluster_list)
print('End Classifying')

Start Classifying
End Classifying


In [27]:
points_of_signs = sign_manager.make_sign_point_dataframe(sign_manager.sign_list)
points_of_signs

,ID,Easting,Northing,Altitude,Retro,Angle,Distance,UTC,Long,Lat
0,16208060.0,754987.929,3741747.458,304.342,0.761,0.52081,17.213,62422.19436,-84.246153,33.785187
1,16208061.0,754987.928,3741747.442,304.148,0.780,0.51052,17.132,62422.19440,-84.246153,33.785186
2,16208062.0,754987.923,3741747.410,303.966,0.776,0.50023,17.073,62422.19443,-84.246153,33.785186
3,16208063.0,754987.922,3741747.399,303.773,0.784,0.48994,16.993,62422.19446,-84.246153,33.785186
4,16208064.0,754987.918,3741747.367,303.593,0.792,0.47966,16.939,62422.19450,-84.246153,33.785186
...,...,...,...,...,...,...,...,...,...,...
36,17920359.0,756524.717,3736225.457,281.053,0.816,0.04761,24.800,62670.87375,-84.231174,33.735069
37,17920360.0,756524.709,3736225.438,280.798,0.816,0.03732,24.815,62670.87378,-84.231174,33.735069
38,17920453.0,756524.850,3736225.460,281.045,0.816,0.04771,24.457,62670.88706,-84.231172,33.735069
39,17920454.0,756524.846,3736225.449,280.794,0.788,0.03742,24.463,62670.88709,-84.231172,33.735069
